<a href="https://colab.research.google.com/github/CarlOsito16/Hexamind/blob/main/Summarization/Hexamind_resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall transformers

Found existing installation: transformers 4.26.0
Uninstalling transformers-4.26.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.8/dist-packages/transformers-4.26.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.26.0


In [38]:
!pip install transformers[sentencepiece]
!pip install gradio
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=b7b2dd60d2214d3c5584f24089cfcf6894634de1d6a1288bd57d4e5d8d85bdf3
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


# Text Summarization

In [1]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM
import re
import gradio as gr

device = 'cuda' if torch.cuda.is_available() else 'cpu'
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """\"Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."""


cmb_ckpt = 'mrm8488/camembert2camembert_shared-finetuned-french-summarization'
cmb_tokenizer = RobertaTokenizerFast.from_pretrained(cmb_ckpt)
cmb_model = EncoderDecoderModel.from_pretrained(cmb_ckpt).to(device)


t5_model_name = "csebuetnlp/mT5_multilingual_XLSum"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)




def clear_input():
    return ("", "")

    
def camembert_generate_summary(article_text):
   inputs = cmb_tokenizer([article_text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = cmb_model.generate(input_ids, attention_mask=attention_mask)
   return cmb_tokenizer.decode(output[0], skip_special_tokens=True)


def t5_generate_summary(article_text):
    input_ids = t5_tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512)["input_ids"]
    
    output_ids = t5_model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]
    
    output = t5_tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
    )

    return output

def summarizer(dropdown_model, article_text):
    """
    Ruturs a summarized version from the full article based on the selected pretrained-model
    """

    if dropdown_model == 'camembert':
        summary = camembert_generate_summary(article_text)

    elif dropdown_model == 'T5':
        summary = t5_generate_summary(article_text)

    return summary



The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
t5_generate_summary(article_text)

'Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin.'

In [10]:
summarizer('T5', article_text)

'Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin.'

In [3]:
import gradio as gr

## Demo: on text summarization

In [28]:
with gr.Blocks() as demo:
    gr.Markdown("Summarize the article text.")
    with gr.Tab("Each model"):
        with gr.Row():
            with gr.Column():
                input_models = gr.Dropdown(['camembert', 'T5'])
                input_article = gr.TextArea(label = 'Article to be summarized')
            with gr.Column():
                summarized_output = gr.TextArea(label= 'Generated summa')
        with gr.Row():

            clear_button = gr.Button("Clear")
            summarize_button = gr.Button("Summarize!")
            
    summarize_button.click(summarizer,
                           inputs = [input_models, input_article] ,
                           outputs = summarized_output)
    
    clear_button.click(clear_input,
                       outputs = [input_models, input_article])
    
    example = "Un nuage de fumée juste après l’explosion, le 1er juin 2019. Une déflagration dans une importante usine d’explosifs du centre de la Russie a fait au moins 79 blessés samedi 1er juin. L’explosion a eu lieu dans l’usine Kristall à Dzerzhinsk, une ville située à environ 400 kilomètres à l’est de Moscou, dans la région de Nijni-Novgorod. « Il y a eu une explosion technique dans l’un des ateliers, suivie d’un incendie qui s’est propagé sur une centaine de mètres carrés », a expliqué un porte-parole des services d’urgence. Des images circulant sur les réseaux sociaux montraient un énorme nuage de fumée après l’explosion. Cinq bâtiments de l’usine et près de 180 bâtiments résidentiels ont été endommagés par l’explosion, selon les autorités municipales. Une enquête pour de potentielles violations des normes de sécurité a été ouverte. Fragments de shrapnel Les blessés ont été soignés après avoir été atteints par des fragments issus de l’explosion, a précisé une porte-parole des autorités sanitaires citée par Interfax. « Nous parlons de blessures par shrapnel d’une gravité moyenne et modérée », a-t-elle précisé. Selon des représentants de Kristall, cinq personnes travaillaient dans la zone où s’est produite l’explosion. Elles ont pu être évacuées en sécurité. Les pompiers locaux ont rapporté n’avoir aucune information sur des personnes qui se trouveraient encore dans l’usine."
    examples = gr.Examples(examples=[ ["camembert",example],
                                     ["T5",example]],
                                     inputs=[input_models, input_article])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://896f3e76-0b3b-4ed1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 64 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7867 <> https://896f3e76-0b3b-4ed1.gradio.live


# Keyword Extraction

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
article_text = "Je peux pas mettre zéro mais sinon j'aurai mis cette note. Depuis décembre j'attends le remboursement d'un ordinateur portable qui n'a jamais été livré. A la place une cafetière. Une semaine pour avoir un bon de retour. Le colis a bien été remis à l'entrepôt et après 10 appels des échanges de mail aujourd'hui on vient de me dire que je ne serai pas remboursée. Je comprends pas la blague. On ne me rembourse pas pour des erreurs d'argent de chez Carrefour. Je ne peux pas travailler de la maison car je n'ai pas d'ordinateur et je peux pas en racheter un car j'ai pas l'argent enfin di il est sur les comptes de carrefour. Je peux porter plainte? Numero de commande 607275144"
n_gram_range = (1, 1)

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range).fit([article_text])
candidates = count.get_feature_names_out()

In [29]:
print(candidates)
print(len(candidates))


['10' '607275144' 'ai' 'appels' 'après' 'argent' 'attends' 'aujourd'
 'aurai' 'avoir' 'bien' 'blague' 'bon' 'cafetière' 'car' 'carrefour'
 'cette' 'chez' 'colis' 'commande' 'comprends' 'comptes' 'de' 'depuis'
 'des' 'di' 'dire' 'décembre' 'en' 'enfin' 'entrepôt' 'erreurs' 'est' 'et'
 'hui' 'il' 'jamais' 'je' 'la' 'le' 'les' 'livré' 'mail' 'mais' 'maison'
 'me' 'mettre' 'mis' 'ne' 'note' 'numero' 'on' 'ordinateur' 'pas' 'peux'
 'place' 'plainte' 'portable' 'porter' 'pour' 'que' 'qui' 'racheter'
 'rembourse' 'remboursement' 'remboursée' 'remis' 'retour' 'semaine'
 'serai' 'sinon' 'sur' 'travailler' 'un' 'une' 'vient' 'zéro' 'échanges'
 'été']
79


In [7]:
import requests
from pathlib import Path

if Path('french_stopword.txt').is_file():
    print('already exists')
else:
    print('not existed yet')
    request = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-fr/master/stopwords-fr.txt")
    with open('french_stopword.txt', "wb") as f:
        f.write(request.content)

not existed yet


In [30]:
# opening the file in read mode
my_file = open("french_stopword.txt", "r")
  
# reading the file
data = my_file.read()
  
# replacing end of line('/n') with ' ' and
# splitting the text it further when '.' is seen.
french_stopwords_list = data.replace('\n', ' ').split(" ")
  
# # printing the data
print(french_stopwords_list)
my_file.close()

['a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'aient', 'aies', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 'aucun', 'aucune', 'aucuns', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autant', 'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'b', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'bon', 'boum', 'bravo', 'brrr', 'c', 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 'celui', 'celui-ci', 'celui-là', 'celà', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 'cet', 'cette', 'ce

In [34]:
article_text = "Je peux pas mettre zéro mais sinon j'aurai mis cette note. Depuis décembre j'attends le remboursement d'un ordinateur portable qui n'a jamais été livré. A la place une cafetière. Une semaine pour avoir un bon de retour. Le colis a bien été remis à l'entrepôt et après 10 appels des échanges de mail aujourd'hui on vient de me dire que je ne serai pas remboursée. Je comprends pas la blague. On ne me rembourse pas pour des erreurs d'argent de chez Carrefour. Je ne peux pas travailler de la maison car je n'ai pas d'ordinateur et je peux pas en racheter un car j'ai pas l'argent enfin di il est sur les comptes de carrefour. Je peux porter plainte? Numero de commande 607275144"
n_gram_range = (1, 1)

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range,
                        stop_words = french_stopwords_list).fit([article_text])
candidates_without_stopwords = count.get_feature_names_out()

In [35]:
len(candidates_without_stopwords)

39

In [36]:
candidates_without_stopwords

array(['10', '607275144', 'appels', 'argent', 'attends', 'blague',
       'cafetière', 'carrefour', 'colis', 'commande', 'comprends',
       'comptes', 'di', 'décembre', 'entrepôt', 'erreurs', 'jamais',
       'livré', 'mail', 'maison', 'mettre', 'mis', 'note', 'numero',
       'ordinateur', 'place', 'plainte', 'portable', 'porter', 'racheter',
       'rembourse', 'remboursement', 'remboursée', 'remis', 'semaine',
       'travailler', 'vient', 'zéro', 'échanges'], dtype=object)

In [42]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([article_text])
candidate_embeddings = model.encode(candidates_without_stopwords)

In [52]:

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates_without_stopwords[index] for index in distances.argsort()[0][-top_n:]]


In [53]:
keywords

['remboursée', 'jamais', '10', 'décembre', 'cafetière']

In [54]:
from sentence_transformers import SentenceTransformer
model =  SentenceTransformer("dangvantuan/sentence-camembert-large")



In [55]:
doc_embedding = model.encode([article_text])
candidate_embeddings = model.encode(candidates_without_stopwords)

In [56]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates_without_stopwords[index] for index in distances.argsort()[0][-top_n:]]

In [57]:
keywords

['livré', 'attends', 'remboursée', 'remboursement', 'rembourse']